# Company Research with AutoGen

In this lesson, we'll learn how to create a team of AI agents to perform company research and analysis using AutoGen. We'll build a system that combines web search, stock analysis, and report generation capabilities.

First, let's set up our environment with the required dependencies:

In [1]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Set required API keys
# _set_env("GOOGLE_SEARCH_ENGINE_ID")
# _set_env("GOOGLE_API_KEY")
_set_env("OPENAI_API_KEY")

In [2]:
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("Async environment configured for Jupyter.")
except ImportError:
    print("Please install nest_asyncio with `pip install nest_asyncio`")

Async environment configured for Jupyter.


In [ ]:
# Import required libraries
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient

Let's create our research tools. First, a web search function using Google's Custom Search API:

In [ ]:
def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:
    import requests
    from bs4 import BeautifulSoup
    import time
    
    url = "https://customsearch.googleapis.com/customsearch/v1"
    params = {
        "key": os.environ["GOOGLE_API_KEY"],
        "cx": os.environ["GOOGLE_SEARCH_ENGINE_ID"],
        "q": query,
        "num": num_results
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        raise Exception(f"Error in API request: {response.status_code}")
        
    results = response.json().get("items", [])
    
    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            return text[:max_chars]
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""
            
    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append({
            "title": item["title"],
            "link": item["link"], 
            "snippet": item["snippet"],
            "body": body
        })
        time.sleep(1)
        
    return enriched_results

Next, let's create a stock analysis function using yfinance:

In [ ]:
def analyze_stock(ticker: str) -> dict:
    import yfinance as yf
    import matplotlib.pyplot as plt
    import numpy as np
    from datetime import datetime, timedelta
    from pytz import timezone
    
    stock = yf.Ticker(ticker)
    
    # Get 1 year of historical data
    end_date = datetime.now(timezone("UTC"))
    start_date = end_date - timedelta(days=365)
    hist = stock.history(start=start_date, end=end_date)
    
    if hist.empty:
        return {"error": "No data available"}
        
    # Calculate metrics
    current_price = hist["Close"].iloc[-1]
    year_high = hist["High"].max()
    year_low = hist["Low"].min()
    ma_50 = hist["Close"].rolling(window=50).mean().iloc[-1]
    ma_200 = hist["Close"].rolling(window=200).mean().iloc[-1]
    
    # Calculate YTD metrics
    ytd_start = datetime(end_date.year, 1, 1, tzinfo=timezone("UTC"))
    ytd_data = hist.loc[ytd_start:]
    if not ytd_data.empty:
        price_change = ytd_data["Close"].iloc[-1] - ytd_data["Close"].iloc[0]
        percent_change = (price_change / ytd_data["Close"].iloc[0]) * 100
    
    # Generate plot
    plt.figure(figsize=(12, 6))
    plt.plot(hist.index, hist["Close"], label="Price")
    plt.plot(hist.index, hist["Close"].rolling(50).mean(), label="50-day MA")
    plt.plot(hist.index, hist["Close"].rolling(200).mean(), label="200-day MA")
    plt.title(f"{ticker} Stock Price")
    plt.xlabel("Date")
    plt.ylabel("Price ($)")
    plt.legend()
    
    plot_path = f"stock_{ticker}.png"
    plt.savefig(plot_path)
    plt.close()
    
    return {
        "current_price": current_price,
        "52_week_high": year_high,
        "52_week_low": year_low,
        "50_day_ma": ma_50,
        "200_day_ma": ma_200,
        "ytd_change": percent_change,
        "plot_path": plot_path
    }

Now let's create our team of agents:

In [ ]:
# Create function tools
search_tool = FunctionTool(google_search, 
                          description="Search Google for company information")
stock_tool = FunctionTool(analyze_stock,
                         description="Analyze stock data and create price chart")

# Create agents
search_agent = AssistantAgent(
    name="Search_Agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    tools=[search_tool],
    system_message="You search for company information and summarize findings."
)

stock_agent = AssistantAgent(
    name="Stock_Agent", 
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    tools=[stock_tool],
    system_message="You analyze stock data and interpret technical indicators."
)

report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    system_message="You compile information into comprehensive research reports."
)

# Create team
team = RoundRobinGroupChat(
    agents=[search_agent, stock_agent, report_agent],
    max_turns=3
)

Let's run our research team:

In [ ]:
# Run the research team
async def research_company(company_name: str):
    stream = team.run_stream(
        task=f"Research and create a report on {company_name}"
    )
    await Console(stream)

# Example usage
await research_company("Tesla")

This will generate a comprehensive report including:
- Company overview from web search
- Stock analysis with technical indicators
- Price chart visualization
- Summary of recent developments
- Key financial metrics

The agents work sequentially:
1. Search Agent gathers information
2. Stock Agent analyzes market data
3. Report Agent compiles everything into a final report

This demonstrates how AutoGen can be used to create specialized agent teams that work together to accomplish complex research tasks.